<a href="https://colab.research.google.com/github/nateraw/modelcards/blob/main/modelcards_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install modelcards
! git config --global credential.helper store

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 86 kB 2.0 MB/s 
     |████████████████████████████████| 596 kB 10.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Loading Existing Model Cards from the Hugging Face Hub 🤗

To load an existing modelcard from a repo on the Hugging Face Hub, like [nateraw/food](https://hf.co/nateraw/food), you can do this:

In [ ]:
from modelcards import ModelCard

card = ModelCard.load('nateraw/food')
print(card)

Downloading:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

---
license: apache-2.0
tags:
- generated_from_trainer
- image-classification
- pytorch
datasets:
- food101
metrics:
- accuracy
model-index:
- name: food101_outputs
  results:
  - task:
      type: image-classification
      name: Image Classification
    dataset:
      type: food101
      name: food-101
      args: default
    metrics:
    - name: Accuracy
      type: accuracy
      value: 0.8912871287128713
---

<!-- This model card has been generated automatically according to the information the Trainer had access to. You
should probably proofread and complete it, then remove this comment. -->

# nateraw/food

This model is a fine-tuned version of [google/vit-base-patch16-224-in21k](https://huggingface.co/google/vit-base-patch16-224-in21k) on the nateraw/food101 dataset.
It achieves the following results on the evaluation set:
- Loss: 0.4501
- Accuracy: 0.8913

## Model description

More information needed

## Intended uses & limitations

More information needed

## Training and ev

We can also see just the metadata like this:

In [ ]:
card.data

license: apache-2.0
tags:
- generated_from_trainer
- image-classification
- pytorch
datasets:
- food101
metrics:
- accuracy
model-index:
- name: food101_outputs
  results:
  - task:
      type: image-classification
      name: Image Classification
    dataset:
      type: food101
      name: food-101
      args: default
    metrics:
    - name: Accuracy
      type: accuracy
      value: 0.8912871287128713

## Creating New Model Cards from the Default Template

This `ModelCard.from_template` is a really nice function. It lets you build a model card from a jinja template. By default, it uses the [default model card template](https://github.com/nateraw/modelcards/blob/main/modelcards/modelcard_template.md), but you could use your own template if you want, which we'll see later :)

To use it, we first define some card metadata using the `modelcards.CardData` class. This is the info that gets stored in the YAML block at the top of the card.

This metadata is leveraged by the Hugging Face Hub to:

- enable discoverability of your model through filters
- provide a standardized way to share your evaluation results (which are then automatically posted to Papers With Code)
- enable the inference API if your model is compatible with one of the available Inference API pipelines.
- And more!

After that, we use the `ModelCard.from_template` fn, passing the `card_data` instance as well as any Jinja template kwargs that were included in our model card template. The default model card template has two Jinja kwargs: `model_id` and `model_description`. 

In [14]:
from huggingface_hub import create_repo
from modelcards import CardData

# ⚠️ Make sure to replace username with your own if you want to push this
repo_id = "nateraw/my-cool-model-with-card"

# Model card metadata leveraged by Hugging Face Hub
card_data = CardData(
    language='en',
    license='mit',
    library_name='timm',
    tags=['image-classification', 'resnet'],
    datasets='beans',
    metrics=['acc', 'f1'],
)

# Instantiate a new card from the default template, supplying card_data and Jinja template kwargs
card = ModelCard.from_template(
    card_data,
    model_id=repo_id.split('/')[-1],  # ⬅️ Jinja template kwargs ⤵️
    model_description="This isn't really a model, it's just a test repo to see if the [modelcards](https://github.com/nateraw/modelcards) package works!",
)
print(card)

---
language: en
license: mit
library_name: timm
tags:
- image-classification
- resnet
datasets: beans
metrics:
- acc
- f1
---

# my-cool-model-with-card

## Model description

This isn't really a model, it's just a test repo to see if the [modelcards](https://github.com/nateraw/modelcards) package works!

## Intended uses & limitations

#### How to use

```python
# You can include sample code which will be formatted
```

#### Limitations and bias

Provide examples of latent issues and potential remediations.

## Training data

Describe the data you used to train the model.
If you initialized it with pre-trained weights, add a link to the pre-trained model card or repository with description of the pre-training data.

## Training procedure

Preprocessing, hardware used, hyperparameters...

## Eval results

Provide some evaluation results.

### BibTeX entry and citation info

```bibtex
@inproceedings{...,
  year={2020}
}
```


You can save your model card to a local file with `ModelCard.save`

In [ ]:
card.save("my_cool_modelcard.md")

We can load it back in using `ModelCard.load`...`ModelCard.load` accepts both repo IDs and local filenames.

In [ ]:
card = ModelCard.load('my_cool_modelcard.md')

In [ ]:
card.data

language: en
license: mit
library_name: timm
tags:
- image-classification
- resnet
datasets: beans
metrics:
- acc
- f1

## Pushing Model Cards to the Hugging Face Hub

As long as you're logged in, you can push model cards to the Hub really easily using the `ModelCard.push_to_hub` function.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


...you can push it up like this

In [ ]:
# Create a repo if it doesn't already exist
repo_url = create_repo(repo_id, exist_ok=True)

# Push To Hub 🤗
card.push_to_hub(repo_id)

print(f"You can find the repo at {repo_url}")

You can find the repo at https://huggingface.co/nateraw/my-cool-model-with-card


## Creating Model Cards From Your Own Templates

As mentioned earlier, the `ModelCard.from_template` fn also allows you to create new cards from a Jinja template that you've defined yourself!

You might find this helpful if you are sharing a ton of models that are quite similar, or if you want to create a template card for a certain task/library/etc.

### Defining our custom template

First, we'll need to define our template. Here, we make a really simple one for example purposes and write it to a file called `my_template.md`.

> ⚠️ **Note** - It's important that you include the `card_data` Jinja template variable within the `---` block at the top, as that's how the `card_data` you define gets included in the README.



In [ ]:
from pathlib import Path

template_text = """
---
{{ card_data }}
---

# {{ model_id | default("CoolModel") }}

This model is part of `super_cool_models` package (which doesn't exist)! It is a fine tuned `cool-model` on the `{{ dataset_name }}`.

## Intended uses & limitations

This model doesn't exist, so you probably don't want to use it! This is just an example template. Please write a very thoughtful model card ❤️
"""

Path('my_template.md').write_text(template_text)

377

### Filling in our custom template

Now we can fill in the template by supplying `template_path='my_template.md'` to `ModelCard.from_template`. Notice how we can provide whatever Jinja template variables we want here...in this case, we'll include `model_id` and `dataset_name`, as we included them in our template.

In [ ]:
card = ModelCard.from_template(
    card_data=CardData(  # Card metadata object that will be converted to YAML block
        language='en',
        license='mit',
        library_name='super_cool_models',
        tags=['image-classification', 'cool-model'],
        datasets='awesome-dataset',
        metrics=['accuracy', 'f1'],
    ),
    template_path='my_template.md', # The template we just wrote!
    model_id='cool-model',  # Jinja template kwarg
    dataset_name='awesome-dataset', # Jinja template kwarg
)
print(card)

---
language: en
license: mit
library_name: super_cool_models
tags:
- image-classification
- cool-model
datasets: awesome-dataset
metrics:
- accuracy
- f1
---

# cool-model

This model is part of `super_cool_models` package (which doesn't exist)! It is a fine tuned `cool-model` on the `awesome-dataset`.

## Intended uses & limitations

This model doesn't exist, so you probably don't want to use it! This is just an example template. Please write a very thoughtful model card ❤️


## Including Evaluation Results in Your Model Card

One of the things the Model Card metadata is used for is to share evaluation results of your model. This is really nice because once you share your results, your model will automatically be shared on the relevant [Papers With Code](https://paperswithcode.com) Leaderboard.

To share evaluation results, you can use the `modelcards.EvalResult` object to define a single evaluation metric for a single dataset. To share multiple metrics (or to share the same metric across multiple datasets), you just create a list of these. These evaluation results are then provided to the `CardData` object upon instantiation.

> ⚠️ **Note** - When providing EvalResults, you must also provide a `model_name` to `CardData`, as it is the name that will be used on PapersWithCode's leaderboard.


Here's what it looks like:

In [ ]:
from modelcards import CardData, EvalResult, ModelCard

model_name = 'my-cool-model-with-eval-results'
card_data=CardData(
    language='en',
    license='mit',
    library_name='timm',
    tags=['image-classification', 'resnet'],
    datasets='beans',
    metrics=['accuracy', 'f1'],
    eval_results=[
        EvalResult(
            task_type='image-classification',
            dataset_type='beans',
            dataset_name='Beans',
            metric_type='accuracy',
            metric_value=0.85,
        ),
        EvalResult(
            task_type='image-classification',
            dataset_type='beans',
            dataset_name='Beans',
            metric_type='f1',
            metric_value=0.75, 
        )
    ],
    model_name=model_name
)

card = ModelCard.from_template(
    card_data,
    model_id=model_name,  # Jinja template kwarg
    model_description="This isn't really a model, it's just a test repo to see if the [modelcards](https://github.com/nateraw/modelcards) package works!"
)

Now lets push that up to the hub to see if it worked!

In [ ]:
# ⚠️ Make sure to replace username below with your own if you're pushing it
repo_id = f"nateraw/{model_name}"

# Create a repo if it doesn't already exist
repo_url = create_repo(repo_id, exist_ok=True)

# Push To Hub 🤗
card.push_to_hub(repo_id)

print(f"You can find the repo at {repo_url}")

You can find the repo at https://huggingface.co/nateraw/my-cool-model-with-eval-results
